In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load data
test_data = pd.read_csv(r"MeDAL\pretrain_subset\test.csv")
train_data = pd.read_csv(r"MeDAL\pretrain_subset\train.csv")
valid_data = pd.read_csv(r"MeDAL\pretrain_subset\valid.csv")

# Save as Parquet (avoids pickling problem)
test_data.to_parquet("test.parquet", engine="pyarrow")
train_data.to_parquet("train.parquet", engine="pyarrow")
valid_data.to_parquet("valid.parquet", engine="pyarrow")

# Load directly into HuggingFace Dataset
from datasets import Dataset

test_dataset = Dataset.from_parquet("test.parquet")
train_dataset = Dataset.from_parquet("train.parquet")
valid_dataset = Dataset.from_parquet("valid.parquet")

# Combine into DatasetDict
from datasets import DatasetDict
medal_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "valid": valid_dataset
})


In [ ]:
medal_dataset["train"]

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
medal_dataset.push_to_hub(
    "lutful2004/MeDAL-dataset",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")

In [ ]:
# Save the dataset dictionary to disk in a folder called 'medal_dataset'
medal_dataset.save_to_disk("medal_dataset")
print("Dataset saved to 'medal_dataset' folder")

In [ ]:
# import os

# # Create a directory to store smaller dataframes if it doesn't exist
# os.makedirs('smaller_datasets', exist_ok=True)

# # Take first 1000 rows from full_data
# full_data_small = full_data.head(1000)

# # Take first 10000 rows from train_data
# train_data_small = train_data.head(10000)

# # Take first 1000 rows from test_data and valid_data
# test_data_small = test_data.head(1000)
# valid_data_small = valid_data.head(1000)

# # Save the smaller dataframes to CSV files
# full_data_small.to_csv('smaller_datasets/full_data_small.csv', index=False)
# train_data_small.to_csv('smaller_datasets/train_data_small.csv', index=False)
# test_data_small.to_csv('smaller_datasets/test_data_small.csv', index=False)
# valid_data_small.to_csv('smaller_datasets/valid_data_small.csv', index=False)

# print("Smaller datasets saved to 'smaller_datasets' folder")

## Samall Sampling

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk


In [2]:
dataset_path = "medal_dataset"
dataset = load_from_disk(dataset_path)

# dataset = load_dataset("lutful2004/MeDAL-dataset")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 3000000
    })
    test: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 1000000
    })
    valid: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 1000000
    })
})

In [6]:
# Process and reduce dataset size to 20% for faster training
train_split = (
    dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * 0.2)))
)
test_split = (
    dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * 0.2)))
)
# The key is "validation" not "valid"
valid_split = (
    dataset["valid"]
    .shuffle(seed=42)
    .select(range(int(len(dataset["valid"]) * 0.2)))
)

dataset_small=DatasetDict()
# Update dataset with smaller splits
dataset_small["train"] = train_split
dataset_small["test"] = test_split
dataset_small["valid"] = valid_split  # Add a "valid" key for easier reference later

In [7]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
dataset_small.push_to_hub(
    "lutful2004/MeDAL-dataset-small",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

KeyboardInterrupt: 